In [6]:
from urllib.request import urlopen  
from tqdm import tqdm
from lxml import html
from datetime import datetime
from collections import Counter
import json
import requests
import time
import re
# from screeninfo import get_monitors
import pandas as pd
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# US

In [14]:
# followers_US = pd.read_csv('Influencers_US_final.csv')
# followers_US.head()

followers_US = []
with open('followers_US.txt', 'r') as f:
    for line in f:
        if "@" in line:
            follower = line.strip()[1:]
            print(follower)
            followers_US.append(follower)
f.close()

followers_US


RevBlackNetwork
TheDemocrats
msnbc
FoxNews
TheAtlantic
aldotcom
JudiciaryGOP
MSNBC
NBCNews
RandPaul
politico
joncoopertweets
HomelandDems
HouseJudiciary
RepSwalwell
NPR
CNN
LPMisesCaucus
RealSpikeCohen
YATPOfficial
justinamash
tedlieu
LPNational
ToddHagopian
theintercept


['RevBlackNetwork',
 'TheDemocrats',
 'msnbc',
 'FoxNews',
 'TheAtlantic',
 'aldotcom',
 'JudiciaryGOP',
 'MSNBC',
 'NBCNews',
 'RandPaul',
 'politico',
 'joncoopertweets',
 'HomelandDems',
 'HouseJudiciary',
 'RepSwalwell',
 'NPR',
 'CNN',
 'LPMisesCaucus',
 'RealSpikeCohen',
 'YATPOfficial',
 'justinamash',
 'tedlieu',
 'LPNational',
 'ToddHagopian',
 'theintercept']

In [8]:
# followers_US_list = followers_US.Influencer_Xhandle.tolist()

count = Counter(followers_US)

followers_US = []

for key, value in count.items():
    if value > 2:
        followers_US.append(key)
        
followers_US

['CNN',
 'TheAtlantic',
 'amyklobuchar',
 'POLITICO',
 'FoxNews',
 'UN',
 'aldotcom',
 'foxnews',
 'WSJ',
 'JulianCastro',
 'RepRaskin',
 'OversightDems',
 'NRSC',
 'gmail',
 'MSNBC',
 'BarackObama',
 'MikeBloomberg',
 'CoryBooker',
 'seanhannity',
 'TenthAmendment',
 'GOP',
 'NHGOP',
 'ewarren',
 'CFPB',
 'FBI',
 'SenWarren',
 'DonaldJTrumpJr',
 'IvankaTrump',
 'SenSanders',
 'BernieSanders',
 'Booker4KY',
 'HoodtotheHoller',
 'brotherjones_',
 'JoeBiden',
 'KamalaHarris',
 'NPR',
 'npr',
 'ABC',
 'davidhogg111',
 'Newsweek',
 'msnbc',
 'WedgeSquared',
 'AOC',
 'LifeNewsHQ',
 'RandPaul',
 'SenTedCruz',
 'realDonaldTrump',
 'NBCNews',
 'AnnCoulter',
 'BillOReilly',
 'NoSpinNews',
 'TeamTrump',
 'EricTrump',
 'Trump',
 'LaraLeaTrump',
 'StJude',
 'TrumpWinery',
 'DanForTexas',
 'Jaybefaunt',
 'RevBlackNetwork',
 '_Jaybefaunt',
 'SteveSchmidtSES',
 'HuffPost',
 'CPRNews',
 'MsLaToshaBrown',
 'BlackVotersMtr',
 'nytimes',
 'washingtonpost',
 'DailyCaller',
 'MichelleObama',
 'HillaryClint

In [9]:
tweets_dict_US = {}

In [10]:
driver = webdriver.Chrome()
driver.get('https://twitter.com/i/flow/login')
wait = WebDriverWait(driver, 10)


In [11]:
### Log in

# Username
username = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[autocomplete=username]'))
)
username.send_keys("rigautam@udel.edu")
time.sleep(3)
login_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '[role=button].r-13qz1uu'))
)
login_button.click()
time.sleep(3)

try: 
    # Password
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[type=password]'))
    )
    password.send_keys("Sus1234!")
    time.sleep(3)
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid*=Login_Button]'))
    )
    login_button.click()
    time.sleep(3)
    
except: # if we get an error
    username = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextTextInput]"))
    )
    username.send_keys("RichaGotem")
    time.sleep(3)
    
    next_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextNextButton]"))
    )
    next_button.click()
    time.sleep(3)
    
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[name*=password]"))
    )
    password.send_keys("Sus1234!")
    time.sleep(3)
    
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=LoginForm_Login_Button]"))
    )
    login_button.click()
    time.sleep(3)
    

In [12]:
# Function to check if the Fetch More button is present
def is_element_present(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.is_displayed()
    except NoSuchElementException:
        return False

count = 0

for profile in tqdm(followers_US):
    
    if profile not in tweets_dict_US.keys():
        print(profile)

        tweets_dict_US[profile] = []

        ### Begin scrape
        time.sleep(3)

        # Search
        search_button = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search and explore"]'))
        )
        search_button.click()
        time.sleep(3)

        search_bar = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search"]'))
        )
        search_bar.click()
        time.sleep(3)

        search_query = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search query"]'))
        )
        # Advanced search query
        search_query.send_keys(f'(Gaza OR Gazan OR Gazans OR Israel OR Israeli OR Israelis OR West OR Bank OR Tel OR Aviv OR Tel OR Aviv-Yafo OR Benduoin OR Benduoins OR Palestine OR Palestinian OR Palestinians OR Middle OR East OR Middle OR Eastern OR Jewish OR Jews OR rabbi OR Muslim OR Muslims OR Islamic OR Jihad OR Islamist OR Islamists OR Antisemitism OR Antisemistic OR Zionism OR Zionist OR Islamophobia OR IDF OR Hamas OR Massacre OR Genocide OR Cease-fire OR Terrorism OR Terrorist OR Terrorists OR Netanyahu OR Histadrut OR Haniyeh OR Yahya OR Sinwar OR Fatah OR Mohammed OR Deif) (# OR #IsraelPalestineWar OR #IsraelGazaWar OR #Gazabombing OR #CeasefireNOW OR #FreePalestine OR #StrikeForGaza OR #Gazagenocide) (from:{profile}) min_faves:1000 until:2024-09-14 since:2023-10-07 -filter:links -filter:replies')

        search_query.send_keys(Keys.RETURN)
        time.sleep(6)
        
        # Scroll until you cannot scroll anymore
        reached_page_end = False
        last_height = driver.execute_script("return document.body.scrollHeight")

        retry_xpath = "//button[@role='button' and @type='button']//span[text()='Retry']"
        # random.randrange(10, 15)
        if (is_element_present(driver, retry_xpath)):
            for i in range(5):
                search_button = wait.until(EC.element_to_be_clickable((By.XPATH, retry_xpath)))
                search_button.click()
                print(f'{profile} - {count} - Retry button appeared.')
            time.sleep(10) 

        while not reached_page_end:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")   
            time.sleep(3)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if last_height == new_height:
                reached_page_end = True
            else:
                last_height = new_height
        try:
            # Select the home timeline div
            search_results = driver.find_element(By.CSS_SELECTOR, '[aria-label="Timeline: Search timeline"]')
            # Just get all the HTML, will parse later
            tweets_listed = search_results.get_attribute('innerHTML')
            time.sleep(3)

            # Append to dict
            tweets_dict_US[profile].append(tweets_listed)
        except:
            pass

        count+=1



  0%|          | 0/317 [00:00<?, ?it/s]

CNN


  0%|          | 1/317 [00:28<2:29:01, 28.30s/it]

TheAtlantic


  1%|          | 2/317 [01:02<2:46:51, 31.78s/it]

amyklobuchar


  1%|          | 3/317 [01:43<3:08:18, 35.98s/it]

POLITICO


  1%|          | 3/317 [01:49<3:11:21, 36.57s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.139)
Stacktrace:
0   chromedriver                        0x0000000103b63338 chromedriver + 5096248
1   chromedriver                        0x0000000103b5ab6a chromedriver + 5061482
2   chromedriver                        0x00000001036e5fd0 chromedriver + 389072
3   chromedriver                        0x00000001036bd3ef chromedriver + 222191
4   chromedriver                        0x000000010375efcd chromedriver + 884685
5   chromedriver                        0x0000000103773c49 chromedriver + 969801
6   chromedriver                        0x0000000103756c93 chromedriver + 851091
7   chromedriver                        0x0000000103725c79 chromedriver + 650361
8   chromedriver                        0x000000010372649e chromedriver + 652446
9   chromedriver                        0x0000000103b260b0 chromedriver + 4845744
10  chromedriver   

In [ ]:
driver.quit()

In [ ]:
tweets_dict_US.keys()

In [ ]:
tweets_dict_US_cleaned = {}
    
for key, value in tweets_dict_US.items():
    if key not in tweets_dict_US_cleaned.keys():
        tweets_dict_US_cleaned[key] = []
        for v in value:
            soup = BeautifulSoup(v, 'html.parser')
            tweets = soup.find_all(attrs={"data-testid": "tweetText"})
            for tweet in tweets:
                tweet_text = tweet.get_text(strip=True)
                tweets_dict_US_cleaned[key].append(tweet_text)

tweets_dict_US_cleaned.keys()

In [ ]:
with open("US_tweets.json", 'w') as file:
    json.dump(tweets_dict_US, file)

In [ ]:
with open("US_tweets_cleaned.json", 'w') as file:
    json.dump(tweets_dict_US_cleaned, file)